# Let's get this party started

In [1]:
#Dependencies
import os
import numpy as np
import pandas as pd
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
import re

# 1. Framing the Problem & Big Picture look (see readme file)

# 2. Get the Data

In [2]:
#set path to datasets
path = os.getcwd()
os.chdir("../datasets/")
filename = "sampledata.xlsx"
ser_path = "./serialized/"

In [3]:
#Importing company datasets
#prod_hist = pd.read_excel(filename, sheet_name="production_hist")
#ship_hist = pd.read_excel(filename, sheet_name="shipment_hist")
#atp_hist = pd.read_excel(filename, sheet_name="atp_hist")
#inventory = pd.read_excel(filename, sheet_name="inventory")

In [4]:
#setting file names
# data = [inventory, atp_hist, prod_hist, ship_hist]
# fnames = ["inventory", "atp_hist", "prod_hist","ship_hist"]
# count = 0
# for df in data:
#     df.name = fnames[count]
#     count+=1

In [5]:
#saving dataframes into pickle files
# for df in data:
#     pklfile = df.name + ".pkl"
#     df.to_pickle(ser_path + pklfile)

#saving dataframes into feather (not working)
for df in data:
    feafile = df.name
    df.to_feather(ser_path+feafile)

In [6]:
#see pklfile name order
pklfiles = os.listdir("./serialized")
pklfiles

['ship_hist.pkl', 'inventory.pkl', 'atp_hist.pkl', 'prod_hist.pkl']

In [7]:
#Fast Upload using pickle files
inventory = pd.read_pickle(ser_path+pklfiles[1])
atp_hist = pd.read_pickle(ser_path+pklfiles[2])
prod_hist = pd.read_pickle(ser_path+pklfiles[3])
ship_hist = pd.read_pickle(ser_path+pklfiles[0])

## Cleaning Up

In [8]:
def clean_columns(dataframe):
    for col in dataframe.columns:
        dataframe.rename(columns={col:re.sub(r'([a-z](?=[A-Z])|[A-Z](?=[A-Z][a-z]))', r'\1 ', col)}, inplace=True)
    dataframe.columns = dataframe.columns.str.strip().str.lower().str.replace(" ","_")
    try:
        dataframe["calendar_day"] = dataframe["calendar_day"].astype("datetime64")
    except:
        pass
    try:
        dataframe["snapshot_date"] = dataframe["snapshot_date"].astype("datetime64")
    finally:
        return dataframe.columns

In [9]:
# attempt to get rid of nulls, need to be more specific with fills to not loose valuable insight. 
# data = [inventory, atp_hist, prod_hist, ship_hist]
# for df in data:
#     df = df.fillna(0, inplace=True)

### Inventory

In [10]:
clean_columns(inventory)

Index(['calendar_day', 'plant', 'material', 'maktx', 'ph3', 'ph5', 'ph',
       'material_type', 'tons', 'shape', 'size_for_qual_cert', 'strgr',
       'block_resource', 'work_center', 'atp'],
      dtype='object')

In [11]:
inventory.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 850873 entries, 0 to 850872
Data columns (total 15 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   calendar_day        850873 non-null  datetime64[ns]
 1   plant               850873 non-null  int64         
 2   material            850873 non-null  int64         
 3   maktx               850873 non-null  object        
 4   ph3                 850873 non-null  object        
 5   ph5                 850873 non-null  object        
 6   ph                  850873 non-null  object        
 7   material_type       850873 non-null  object        
 8   tons                850873 non-null  float64       
 9   shape               848401 non-null  object        
 10  size_for_qual_cert  848401 non-null  object        
 11  strgr               850873 non-null  int64         
 12  block_resource      626217 non-null  object        
 13  work_center         850866 no

In [12]:
#Filling in empty atp rows with zero
inventory["atp"].fillna(0, inplace=True)

In [13]:
# inventory["work_center"] = inventory["work_center"].astype("int64")

In [14]:
inventory = inventory.loc[inventory["material_type"]=="ZERT"]
inventory["material_type"].unique()

array(['ZERT'], dtype=object)

In [15]:
inventory_reduced = inventory[["calendar_day","material","maktx", "ph5", "shape", "size_for_qual_cert", "block_resource", "tons", "atp"]]
inventory_reduced.rename(columns={"material":"sku", "maktx":"material_description", "size_for_qual_cert":"size"}, inplace=True)
inventory_reduced.head()

,calendar_day,sku,material_description,ph5,shape,size,block_resource,tons,atp
0,2020-02-23,106022474,"L 2X2X1/4 A36/44W/A529-50 20'00""",LS-Mills-Mer-Mer-Lght Shp,SHAPE_L,2X2X1/4,"2"" ANG",0.009,0.0
1,2020-02-20,106022474,"L 2X2X1/4 A36/44W/A529-50 20'00""",LS-Mills-Mer-Mer-Lght Shp,SHAPE_L,2X2X1/4,"2"" ANG",0.009,0.0
2,2020-02-19,106022474,"L 2X2X1/4 A36/44W/A529-50 20'00""",LS-Mills-Mer-Mer-Lght Shp,SHAPE_L,2X2X1/4,"2"" ANG",0.009,0.0
3,2020-02-18,106022474,"L 2X2X1/4 A36/44W/A529-50 20'00""",LS-Mills-Mer-Mer-Lght Shp,SHAPE_L,2X2X1/4,"2"" ANG",0.009,0.0
4,2020-02-17,106022474,"L 2X2X1/4 A36/44W/A529-50 20'00""",LS-Mills-Mer-Mer-Lght Shp,SHAPE_L,2X2X1/4,"2"" ANG",0.009,0.0


In [16]:
inventory_reduced.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 649986 entries, 0 to 850872
Data columns (total 9 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   calendar_day          649986 non-null  datetime64[ns]
 1   sku                   649986 non-null  int64         
 2   material_description  649986 non-null  object        
 3   ph5                   649986 non-null  object        
 4   shape                 647514 non-null  object        
 5   size                  647514 non-null  object        
 6   block_resource        626217 non-null  object        
 7   tons                  649986 non-null  float64       
 8   atp                   649986 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(5)
memory usage: 49.6+ MB


In [17]:
inventory_reduced[inventory_reduced.isna().any(axis=1)]

,calendar_day,sku,material_description,ph5,shape,size,block_resource,tons,atp
3819,2020-02-23,109004211,"PR 3/8"" 1018 20'00"" CD",LS-Mills-Mer-Mer-MBQ,PENCIL_ROD,"3/8""",NaN,-0.002,0.0
3820,2020-02-20,109004211,"PR 3/8"" 1018 20'00"" CD",LS-Mills-Mer-Mer-MBQ,PENCIL_ROD,"3/8""",NaN,-0.002,0.0
3821,2020-02-19,109004211,"PR 3/8"" 1018 20'00"" CD",LS-Mills-Mer-Mer-MBQ,PENCIL_ROD,"3/8""",NaN,-0.002,0.0
3822,2020-02-18,109004211,"PR 3/8"" 1018 20'00"" CD",LS-Mills-Mer-Mer-MBQ,PENCIL_ROD,"3/8""",NaN,-0.002,0.0
3823,2020-02-17,109004211,"PR 3/8"" 1018 20'00"" CD",LS-Mills-Mer-Mer-MBQ,PENCIL_ROD,"3/8""",NaN,-0.002,0.0
...,...,...,...,...,...,...,...,...,...
850145,2017-03-27,109004278,"F 5/16X1 3/4 A36/44W 20'00""",LS-Mills-Mer-Mer-MBQ,FLAT,5/16 X 1 3/4,NaN,0.001,0.0
850146,2017-04-05,109004278,"F 5/16X1 3/4 A36/44W 20'00""",LS-Mills-Mer-Mer-MBQ,FLAT,5/16 X 1 3/4,NaN,0.001,0.0
850147,2017-02-23,109004278,"F 5/16X1 3/4 A36/44W 20'00""",LS-Mills-Mer-Mer-MBQ,FLAT,5/16 X 1 3/4,NaN,0.001,0.0
850148,2017-03-15,109004278,"F 5/16X1 3/4 A36/44W 20'00""",LS-Mills-Mer-Mer-MBQ,FLAT,5/16 X 1 3/4,NaN,0.001,0.0


### ATP History

In [18]:
clean_columns(atp_hist)

Index(['plant', 'plant_description', 'sku', 'atp_floor', 'snapshot_date'], dtype='object')

In [19]:
atp_hist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235706 entries, 0 to 235705
Data columns (total 5 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   plant              235706 non-null  int64         
 1   plant_description  235706 non-null  object        
 2   sku                235706 non-null  object        
 3   atp_floor          235706 non-null  float64       
 4   snapshot_date      235706 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 9.0+ MB


In [20]:
atp_reduced = atp_hist[["sku", "atp_floor", "snapshot_date"]]
atp_reduced.head()

,sku,atp_floor,snapshot_date
0,102003353,2.0,2019-11-01
1,102003353,2.0,2019-10-17
2,102003353,2.0,2019-10-16
3,102003353,2.0,2019-10-15
4,102003353,2.0,2019-10-14


### Production History

In [21]:
clean_columns(prod_hist)

Index(['calendar_day', 'plant', 'plant_description', 'production_process',
       'production_process_description', 'work_center', 'wc_description',
       'material', 'material_description', 'ph3', 'ph5', 'ton', 'block',
       'shape', 'size', 'stratedgy', 'month/year', 'year', 'monthyear'],
      dtype='object')

In [22]:
prod_hist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41935 entries, 0 to 41934
Data columns (total 19 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   calendar_day                    41935 non-null  datetime64[ns]
 1   plant                           41935 non-null  int64         
 2   plant_description               41935 non-null  object        
 3   production_process              41935 non-null  int64         
 4   production_process_description  41935 non-null  object        
 5   work_center                     37833 non-null  object        
 6   wc_description                  41935 non-null  object        
 7   material                        41935 non-null  int64         
 8   material_description            41935 non-null  object        
 9   ph3                             36579 non-null  object        
 10  ph5                             36579 non-null  object        
 11  to

In [23]:
#Removing strategy 10 (Note: check assumption that these are all billets)
prod_hist = prod_hist.loc[(prod_hist["stratedgy"] !=10)]

In [24]:
# prod_hist["stratedgy"] = prod_hist["stratedgy"].astype("int64")

In [25]:
prod_reduced = prod_hist[["calendar_day", "production_process", "production_process_description","stratedgy","work_center","wc_description","block","shape","size","material","material_description","ton"]]
prod_reduced.rename(columns={"material":"sku", "stratedgy":"strategy"}, inplace=True)
prod_reduced.head()

,calendar_day,production_process,production_process_description,strategy,work_center,wc_description,block,shape,size,sku,material_description,ton
0,2013-02-06,4310,Rolling,63.0,25000001,ROLLING - STRUCTURAL,"4"" ANG",SHAPE_L,4X4X3/8,106023315,"L 4X4X3/8 44W 60'00""",40
1,2013-02-06,4310,Rolling,63.0,25000001,ROLLING - STRUCTURAL,"4"" ANG",SHAPE_L,4X4X3/8,106022771,"L 4X4X3/8 A36/44W/A529-50 40'00""",390
2,2013-02-06,4310,Rolling,63.0,25000001,ROLLING - STRUCTURAL,"4"" ANG",SHAPE_L,4X4X3/8,106022760,"L 4X4X3/8 A36/44W/A529-50 20'00""",345
3,2013-02-06,4310,Rolling,63.0,25000001,ROLLING - STRUCTURAL,"4"" ANG",SHAPE_L,4X4X3/8,106022755,"L 4X4X3/8 55W 60'00""",205
4,2013-02-06,4310,Rolling,63.0,25000001,ROLLING - STRUCTURAL,"4"" ANG",SHAPE_L,4X4X3/8,106022749,"L 4X4X3/8 50W 60'00""",32


In [26]:
#Removing prod_processes up the supply chain: melt, scrap etc.
prod_reduced = prod_reduced.loc[(prod_reduced["production_process_description"] != "Melt shop") & (prod_reduced["production_process_description"] != "Scrap yard")]
prod_reduced = prod_reduced.loc[(prod_reduced["production_process_description"] != "Finishing production")]

In [28]:
prod_reduced.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14822 entries, 0 to 36155
Data columns (total 12 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   calendar_day                    14822 non-null  datetime64[ns]
 1   production_process              14822 non-null  int64         
 2   production_process_description  14822 non-null  object        
 3   strategy                        14820 non-null  float64       
 4   work_center                     13452 non-null  object        
 5   wc_description                  14822 non-null  object        
 6   block                           14815 non-null  object        
 7   shape                           14795 non-null  object        
 8   size                            14795 non-null  object        
 9   sku                             14822 non-null  int64         
 10  material_description            14822 non-null  object        
 11  to

In [29]:
prod_reduced[prod_reduced["block"].isna()]["production_process_description"]

35617    Rolling
35618    Rolling
35631    Rolling
35632    Rolling
36135    Rolling
36154    Rolling
36155    Rolling
Name: production_process_description, dtype: object

In [30]:
#need to figure out how to trace the 30K rows missing in block - maybe get rid of "meltshop, scrap yard, finishing production"

In [31]:
prod_reduced[prod_reduced["block"].isna()].groupby("production_process_description").count()

,calendar_day,production_process,strategy,work_center,wc_description,block,shape,size,sku,material_description,ton
production_process_description,,,,,,,,,,,
Rolling,7,7,5,5,7,0,5,5,7,7,7


In [32]:
prod_reduced[prod_reduced["production_process_description"]=="Melt shop"]

,calendar_day,production_process,production_process_description,strategy,work_center,wc_description,block,shape,size,sku,material_description,ton


### Ship History

In [33]:
clean_columns(ship_hist)

Index(['shipping_day', 'plant', 'plant_description', 'sales_organization',
       'sales_doc_type', 'outside_rep_ship-to', 'inside_rep_ship-to',
       'sold-to_party', 'sold-to_part_name', 'ship-to_party',
       'ship-to_party_name', 'postal_code', 'location', 'region',
       'region_description', 'country', 'country_description', 'material',
       'material_description', 'div_id', 'div', 'prod_hier_level_3',
       'prod_hier_level_5', 'shipment_tons', 'manager', 'segment',
       'plantcountry', 'businessdaycounter', 'block', 'lvorm', 'mrptype',
       'proctype', 'relativemonth', 'shipping_week', 'strgr', 'shape',
       'sizeforqualcert', 'dailyshiprate3', 'monthdelta', 'dailyshiprate6',
       'dailyshiprate12', 'workcenter', 'wc_desc', 'bayou_flag'],
      dtype='object')

In [34]:
ship_hist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156796 entries, 0 to 156795
Data columns (total 44 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   shipping_day          156796 non-null  datetime64[ns]
 1   plant                 156796 non-null  int64         
 2   plant_description     156796 non-null  object        
 3   sales_organization    156796 non-null  object        
 4   sales_doc_type        156796 non-null  object        
 5   outside_rep_ship-to   156796 non-null  object        
 6   inside_rep_ship-to    156796 non-null  object        
 7   sold-to_party         156796 non-null  int64         
 8   sold-to_part_name     156796 non-null  object        
 9   ship-to_party         156796 non-null  int64         
 10  ship-to_party_name    156796 non-null  object        
 11  postal_code           156796 non-null  object        
 12  location              156796 non-null  object        
 13 

In [35]:
#Filling in NaN Shipment field rows to zero
ship_hist["shipment_tons"].fillna(0, inplace=True)

In [36]:
# Attempt to get correct data types, needs work.
# ship_hist["businessdaycounter"] = ship_hist["businessdaycounter"].astype("int64")
# ship_hist["strgr"] = ship_hist["strgr"].astype("int64")
# ship_hist["workcenter"] = ship_hist["workcenter"].astype("int64")

In [37]:
#Reducing main data frame
ship_reduced = ship_hist[["shipping_day", "block", "strgr", "shape", "sizeforqualcert", "div", "material", "material_description","shipment_tons", "segment", "sold-to_party","ship-to_party", "postal_code","region_description", "country"]]
ship_reduced.rename(columns={"material":"sku", "strgr":"strategy","sizeforqualcert":"size"}, inplace=True)
ship_reduced.head()

,shipping_day,block,strategy,shape,size,div,sku,material_description,shipment_tons,segment,sold-to_party,ship-to_party,postal_code,region_description,country
0,2020-01-29,"8"" CHN",40.0,SHAPE_C,8 X 11.5#,Merchants,106021716,"C 8X11.5# GGMULTI 40'00""",0.0,NA - DISTRIBUTION,100092779,100099953,72301-7103,Arkansas,US
1,2020-01-07,"8"" CHN",40.0,SHAPE_C,8 X 11.5#,Merchants,106021716,"C 8X11.5# GGMULTI 40'00""",0.0,NA - DISTRIBUTION,100092779,100099953,72301-7103,Arkansas,US
2,2020-01-24,"8"" CHN",40.0,SHAPE_C,8 X 11.5#,Merchants,106021716,"C 8X11.5# GGMULTI 40'00""",0.0,NA - DISTRIBUTION,100092318,100099655,37218-3813,Tennessee,US
3,2020-01-29,"3 X 2"" ANG",40.0,SHAPE_L,3X2X1/2,Merchants,106022002,"L 3X2X1/2 GGMULTI 40'00""",0.0,NA - DISTRIBUTION,100092779,100099953,72301-7103,Arkansas,US
4,2020-01-27,"12"" CHN",40.0,SHAPE_C,12 X 20.7#,Merchants,106020260,"C 12X20.7# GGMULTI 40'00""",0.0,NA - DISTRIBUTION,100092309,100095766,35954,Alabama,US


In [38]:
#Check unique Divisions
ship_hist["div"].unique()

array(['Merchants', 'Rebar', 'Billets', 'SBQ Bar', 'Scrap'], dtype=object)

In [39]:
#Removing billets and scrap from dataframe
ship_reduced = ship_reduced.loc[(ship_reduced["div"] != "Billets") & (ship_reduced["div"] != "Scrap")]

In [40]:
ship_reduced.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 156711 entries, 0 to 156795
Data columns (total 15 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   shipping_day          156711 non-null  datetime64[ns]
 1   block                 153079 non-null  object        
 2   strategy              156700 non-null  float64       
 3   shape                 156700 non-null  object        
 4   size                  156700 non-null  object        
 5   div                   156711 non-null  object        
 6   sku                   156711 non-null  int64         
 7   material_description  156711 non-null  object        
 8   shipment_tons         156711 non-null  float64       
 9   segment               154025 non-null  object        
 10  sold-to_party         156711 non-null  int64         
 11  ship-to_party         156711 non-null  int64         
 12  postal_code           156711 non-null  object        
 13 

In [41]:
#Show Sample of NaN fields
ship_reduced[ship_reduced.isna().any(axis=1)]

,shipping_day,block,strategy,shape,size,div,sku,material_description,shipment_tons,segment,sold-to_party,ship-to_party,postal_code,region_description,country
91,2020-01-28,"2-1/2"" ANG",40.0,SHAPE_L,2 1/2X2 1/2X1/4,Merchants,106028782,"L 2 1/2X2 1/2X1/4 GGMULTI 20'00""",0.0,NaN,100051158,100051158,77029,Texas,US
152,2019-10-29,"2-1/2"" ANG",40.0,SHAPE_L,2 1/2X2 1/2X1/4,Merchants,106028782,"L 2 1/2X2 1/2X1/4 GGMULTI 20'00""",0.0,NaN,100051158,100051158,77029,Texas,US
314,2020-01-06,NaN,40.0,FLAT,1/4 X 5,Merchants,109005365,"F 1/4X5 GGMULTI 20'00""",0.0,NA - DISTRIBUTION,100091890,100094606,N2R 1E6,Ontario,CA
430,2020-02-04,"8"" FLT",40.0,FLAT,1/2 X 8,Merchants,109008964,"F 1/2X8 GGMULTI 20'00""",0.0,NaN,100092155,100478827,S6V 5P9,Saskatchewan,CA
431,2020-02-04,"2"" ANG",40.0,SHAPE_L,2X2X1/8,Merchants,106029077,"L 2X2X1/8 GGMULTI 20'00""",0.0,NaN,100092155,100478827,S6V 5P9,Saskatchewan,CA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155890,2017-05-24,NaN,40.0,SHAPE_C,15 X 50#,Merchants,106027672,"C 15X50# GGMULTI 40'00""",24.0,NA - DISTRIBUTION,100091238,100103765,14606-3100,New York,US
156012,2017-02-08,NaN,40.0,SHAPE_C,15 X 33.9#,Merchants,106019519,"C 15X33.9# GGMULTI 40'00""",5.0,NA - DISTRIBUTION,100060362,100060377,49444-2333,Michigan,US
156429,2017-03-22,NaN,40.0,SHAPE_C,15 X 33.9#,Merchants,106027715,"C 15X33.9# GGMULTI 50'00""",8.0,NA - DISTRIBUTION,100060244,100060250,40209-1845,Kentucky,US
156455,2017-01-31,NaN,40.0,SHAPE_C,15 X 33.9#,Merchants,106019519,"C 15X33.9# GGMULTI 40'00""",5.0,NA - DISTRIBUTION,100060244,100114121,60440,Illinois,US


In [42]:
#Show the shape that show as null within the block
ship_reduced[ship_reduced["block"].isna()].groupby("shape").count()

,shipping_day,block,strategy,size,div,sku,material_description,shipment_tons,segment,sold-to_party,ship-to_party,postal_code,region_description,country
shape,,,,,,,,,,,,,,
ASDRAWING,2,0,2,2,2,2,2,2,2,2,2,2,2,2
FLAT,2457,0,2457,2457,2457,2457,2457,2457,2249,2457,2457,2457,2457,2457
PENCIL_ROD,81,0,81,81,81,81,81,81,81,81,81,81,81,81
ROUND,246,0,246,246,246,246,246,246,216,246,246,246,246,246
SHAPE_C,237,0,237,237,237,237,237,237,231,237,237,237,237,237
SHAPE_L,415,0,415,415,415,415,415,415,384,415,415,415,415,415
SHAPE_MC,31,0,31,31,31,31,31,31,31,31,31,31,31,31
SQUARE,152,0,152,152,152,152,152,152,139,152,152,152,152,152


### Cleaned Reduced Datasets

In [43]:
data_reduced = [ship_reduced, inventory_reduced, atp_reduced, prod_reduced]


In [44]:
#storing cleaned Prod
%store prod_reduced

#storing cleaned Ship
%store ship_reduced

#storing cleaned Inventory
%store inventory_reduced

#storing cleaned ATP
%store atp_reduced

Stored 'prod_reduced' (DataFrame)
Stored 'ship_reduced' (DataFrame)
Stored 'inventory_reduced' (DataFrame)
Stored 'atp_reduced' (DataFrame)


## Add Sales Order Data

# 3. Exploratory Data Analysis